# GPT Excerpt Replacing

Replace hallucinated excerpts with valid excerpts from re-run questions until all questions have 3 valid excerpts

In [1]:
#Imports libraries
import pandas as pd
import numpy as np

In [2]:
#Read in continuity checked results for run 1
df5_1 = pd.read_csv('/work/oz_extractor_5.1_1_is_match_results_CONT_CHECKED_2.csv')

#Drop questions 'Is Toto in Book 3, Ozma of Oz?', as the question was updated after run 1
df5_1 = df5_1[df5_1['question'] != 'Is Toto in Book 3, Ozma of Oz?']

#Add column for run number
df5_1['run_number'] = 1

df5_1.head()


,question,question_number,result_rank,excerpt,filename,model_chunk_index,model_chapter_number,model_chapter_title,model_chunk_in_chapter_index,all_model_chunks,model_filename,excerpt_cleaned,run_number
0,What color are Dorothy's shoes?,1,1,"“Dorothy looked, and gave a little cry of frig...",book1_the_wonderful_wizard_of_oz_pg55_images,NaN,NaN,NaN,NaN,[],NaN,dorothy looked and gave a little cry of fright...,1
1,What color are Dorothy's shoes?,1,2,"““What is it?” asked the little old woman, and...",book1_the_wonderful_wizard_of_oz_pg55_images,16.0,2.0,The Council With The Munchkins,7.0,"[16, 17]",the_wonderful_wizard_of_oz,what is it? asked the little old woman and loo...,1
2,What color are Dorothy's shoes?,1,3,"“Dorothy had only one other dress, but that ha...",book1_the_wonderful_wizard_of_oz_pg55_images,22.0,3.0,How Dorothy Saved The Scarecrow,0.0,"[22, 23]",the_wonderful_wizard_of_oz,dorothy had only one other dress but that happ...,1
3,How old is the Scarecrow when Dorothy finds him?,2,1,"“She ate a hearty breakfast, and watched a wee...",book1_the_wonderful_wizard_of_oz_pg55_images,27.0,3.0,How Dorothy Saved The Scarecrow,5.0,"[27, 28, 29, 30]",the_wonderful_wizard_of_oz,she ate a hearty breakfast and watched a wee m...,1
4,How old is the Scarecrow when Dorothy finds him?,2,2,"“And Dorothy, with her basket on her arm, once...",book1_the_wonderful_wizard_of_oz_pg55_images,NaN,NaN,NaN,NaN,[],NaN,and dorothy with her basket on her arm once mo...,1


In [3]:
#Read in continuity checked results for run 2
df5_2 = pd.read_csv('/work/oz_extractor_5_2_is_match_results_CONT_CHECKED_1.csv')

#Add column for run number
df5_2['run_number'] = 2

df5_2.head()


,question,question_number,result_rank,excerpt,filename,model_chunk_index,model_chapter_number,model_chapter_title,model_chunk_in_chapter_index,all_model_chunks,model_filename,excerpt_cleaned,run_number
0,What color are Dorothy's shoes?,1,1,"“Dorothy had only one other dress, but that ha...",book1_the_wonderful_wizard_of_oz_pg55_images,22.0,3.0,How Dorothy Saved The Scarecrow,0.0,"[22, 23]",the_wonderful_wizard_of_oz,dorothy had only one other dress but that happ...,2
1,What color are Dorothy's shoes?,1,2,"“Dorothy looked, and gave a little cry of frig...",book1_the_wonderful_wizard_of_oz_pg55_images,12.0,2.0,The Council With The Munchkins,3.0,"[12, 13]",the_wonderful_wizard_of_oz,dorothy looked and gave a little cry of fright...,2
2,What color are Dorothy's shoes?,1,3,“‘Your Silver Shoes will carry you over the de...,book1_the_wonderful_wizard_of_oz_pg55_images,252.0,23.0,Glinda The Good Witch Grants Dorothy’s Wish,5.0,"[252, 253]",the_wonderful_wizard_of_oz,your silver shoes will carry you over the dese...,2
3,How old is the Scarecrow when Dorothy finds him?,2,1,“When she had gone several miles she thought s...,book1_the_wonderful_wizard_of_oz_pg55_images,NaN,NaN,NaN,NaN,[],NaN,when she had gone several miles she thought sh...,2
4,How old is the Scarecrow when Dorothy finds him?,2,2,“Dorothy reached up both arms and lifted the f...,book1_the_wonderful_wizard_of_oz_pg55_images,30.0,3.0,How Dorothy Saved The Scarecrow,8.0,"[30, 31, 32, 33]",the_wonderful_wizard_of_oz,dorothy reached up both arms and lifted the fi...,2


In [4]:
def replace_hallucinated_excerpts(master_df: pd.DataFrame, new_df: pd.DataFrame) -> pd.DataFrame:
    """
    Fills rows in master_df where 'model_filename' is NaN by replacing the entire row
    with an unused matching row from a new run's df.

    Matching is based on the 'question' column, and only non-NaN rows in new_df['model_filename']
    are considered as replacements. Each row in new_df can be used only once.

    Args:
        master_df: The base DataFrame to be modified.
        new_df: The DataFrame containing replacement data.

    Returns:
        The updated master_df.
    """
    # Create copy to make changes on
    master_df = master_df.copy()
    
    # Track the indices of new_df that have been used
    used_indices = set()

    # Iterate over the indices of master_df
    for index in master_df.index:
        # Check if the row needs a replacement (model_filename is NaN)
        if pd.isna(master_df.loc[index, 'model_filename']):
            
            # Get the question for matching
            question_value = master_df.loc[index, 'question']
            
            # Filter new_df for matching question and non-null fill column
            matching_rows = new_df[
                (new_df['question'] == question_value) & 
                new_df['model_filename'].notna()
            ]
            
            # Find the first unused matching row
            for idx, match_row in matching_rows.iterrows():
                if idx not in used_indices:
                    # Replace the entire row in master_df with the replacement row
                    master_df.loc[index] = match_row
                    
                    # Mark this index from new_df as used
                    used_indices.add(idx)
                    break
                    
    return master_df

In [5]:
# Replace hallucinated excerpts in df5_2 with those from df5_1
master_df = replace_hallucinated_excerpts(df5_2, df5_1)
master_df.head()

,question,question_number,result_rank,excerpt,filename,model_chunk_index,model_chapter_number,model_chapter_title,model_chunk_in_chapter_index,all_model_chunks,model_filename,excerpt_cleaned,run_number
0,What color are Dorothy's shoes?,1,1,"“Dorothy had only one other dress, but that ha...",book1_the_wonderful_wizard_of_oz_pg55_images,22.0,3.0,How Dorothy Saved The Scarecrow,0.0,"[22, 23]",the_wonderful_wizard_of_oz,dorothy had only one other dress but that happ...,2
1,What color are Dorothy's shoes?,1,2,"“Dorothy looked, and gave a little cry of frig...",book1_the_wonderful_wizard_of_oz_pg55_images,12.0,2.0,The Council With The Munchkins,3.0,"[12, 13]",the_wonderful_wizard_of_oz,dorothy looked and gave a little cry of fright...,2
2,What color are Dorothy's shoes?,1,3,“‘Your Silver Shoes will carry you over the de...,book1_the_wonderful_wizard_of_oz_pg55_images,252.0,23.0,Glinda The Good Witch Grants Dorothy’s Wish,5.0,"[252, 253]",the_wonderful_wizard_of_oz,your silver shoes will carry you over the dese...,2
3,How old is the Scarecrow when Dorothy finds him?,2,1,"“She ate a hearty breakfast, and watched a wee...",book1_the_wonderful_wizard_of_oz_pg55_images,27.0,3.0,How Dorothy Saved The Scarecrow,5.0,"[27, 28, 29, 30]",the_wonderful_wizard_of_oz,she ate a hearty breakfast and watched a wee m...,1
4,How old is the Scarecrow when Dorothy finds him?,2,2,“Dorothy reached up both arms and lifted the f...,book1_the_wonderful_wizard_of_oz_pg55_images,30.0,3.0,How Dorothy Saved The Scarecrow,8.0,"[30, 31, 32, 33]",the_wonderful_wizard_of_oz,dorothy reached up both arms and lifted the fi...,2


In [6]:
def get_unique_null_questions(df: pd.DataFrame) -> np.ndarray:
    """
    Returns an array of unique question strings where the 'model_filename'
    column is null (NaN). These questions can be used for rerunning the
    extraction process to get more unhallucinated results.

    Args:
        df: A DataFrame containing 'question' and 'model_filename' columns.

    Returns:
        A NumPy array of unique question strings that correspond to null 
        'model_filename' entries.
    """
    
    # Filter for rows where 'model_filename' is null
    null_model_df = df[df['model_filename'].isnull()]
    
    # Return the unique questions from the filtered DataFrame
    return null_model_df['question'].unique()

In [7]:
# Extract questions that need to be rerun
rerun_questions = get_unique_null_questions(master_df)
rerun_questions

array(['How old is the Scarecrow when Dorothy finds him?',
       'When is Wogglebug introduced?', 'When do Dorothy and Ozma meet?',
       'How do you wind Tiktok?', 'What does Eureka look like?',
       'Who is Zeb?',
       'How does Dorothy return to Kansas in book 4, Dorothy and the Wizard in Oz?',
       'What are the Cuttenclips?',
       'In The Emerald City of Oz, what is the first instance of the author speaking in second person? ',
       'How is Mogodore defeated?'], dtype=object)

In [8]:
# Read in continuity checked results for run 3
df5_3 = pd.read_csv('/work/oz_extractor_5_3_is_match_results_CONT_CHECKED_1.csv')

#Add column for run number
df5_3['run_number'] = 3

df5_3.head()

,question,question_number,result_rank,excerpt,filename,model_chunk_index,model_chapter_number,model_chapter_title,model_chunk_in_chapter_index,all_model_chunks,model_filename,excerpt_cleaned,run_number
0,How old is the Scarecrow when Dorothy finds him?,1,1,“When he had gone several miles she thought sh...,book1_the_wonderful_wizard_of_oz_pg55_images,NaN,NaN,NaN,NaN,[],NaN,when he had gone several miles she thought she...,3
1,How old is the Scarecrow when Dorothy finds him?,1,2,"““I am not feeling well,” said the Scarecrow, ...",book1_the_wonderful_wizard_of_oz_pg55_images,NaN,NaN,NaN,NaN,[],NaN,i am not feeling well said the scarecrow with ...,3
2,How old is the Scarecrow when Dorothy finds him?,1,3,““If you will come with me I’ll ask Oz to do a...,book1_the_wonderful_wizard_of_oz_pg55_images,NaN,NaN,NaN,NaN,[],NaN,if you will come with me ill ask oz to do all ...,3
3,"When is the first time we read ""There's no pla...",2,1,“That is because you have no brains” answered ...,book1_the_wonderful_wizard_of_oz_pg55_images,NaN,NaN,NaN,NaN,[],NaN,that is because you have no brains answered th...,3
4,"When is the first time we read ""There's no pla...",2,2,“That is because you have no brains” answered ...,book1_the_wonderful_wizard_of_oz_pg55_images,NaN,NaN,NaN,NaN,[],NaN,that is because you have no brains answered th...,3


In [9]:
# Replace hallucinated excerpts in master_df with those from df5_3
master_df = replace_hallucinated_excerpts(master_df, df5_3)
master_df

,question,question_number,result_rank,excerpt,filename,model_chunk_index,model_chapter_number,model_chapter_title,model_chunk_in_chapter_index,all_model_chunks,model_filename,excerpt_cleaned,run_number
0,What color are Dorothy's shoes?,1,1,"“Dorothy had only one other dress, but that ha...",book1_the_wonderful_wizard_of_oz_pg55_images,22.0,3.0,How Dorothy Saved The Scarecrow,0.0,"[22, 23]",the_wonderful_wizard_of_oz,dorothy had only one other dress but that happ...,2
1,What color are Dorothy's shoes?,1,2,"“Dorothy looked, and gave a little cry of frig...",book1_the_wonderful_wizard_of_oz_pg55_images,12.0,2.0,The Council With The Munchkins,3.0,"[12, 13]",the_wonderful_wizard_of_oz,dorothy looked and gave a little cry of fright...,2
2,What color are Dorothy's shoes?,1,3,“‘Your Silver Shoes will carry you over the de...,book1_the_wonderful_wizard_of_oz_pg55_images,252.0,23.0,Glinda The Good Witch Grants Dorothy’s Wish,5.0,"[252, 253]",the_wonderful_wizard_of_oz,your silver shoes will carry you over the dese...,2
3,How old is the Scarecrow when Dorothy finds him?,2,1,"“She ate a hearty breakfast, and watched a wee...",book1_the_wonderful_wizard_of_oz_pg55_images,27.0,3.0,How Dorothy Saved The Scarecrow,5.0,"[27, 28, 29, 30]",the_wonderful_wizard_of_oz,she ate a hearty breakfast and watched a wee m...,1
4,How old is the Scarecrow when Dorothy finds him?,2,2,“Dorothy reached up both arms and lifted the f...,book1_the_wonderful_wizard_of_oz_pg55_images,30.0,3.0,How Dorothy Saved The Scarecrow,8.0,"[30, 31, 32, 33]",the_wonderful_wizard_of_oz,dorothy reached up both arms and lifted the fi...,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
205,Why doesn't Eureka immediately tell Ozma where...,69,2,"""The piglet that belonged to the Princess wore...",book4_dorothy_and_the_wizard_in_ozpg22566_images,280.0,19.0,The Wizard Performs Another Trick,9.0,"[280, 281, 282, 283]",dorothy_and_the_wizard_in_oz,the piglet that belonged to the princess wore ...,2
206,Why doesn't Eureka immediately tell Ozma where...,69,3,"""As the Princess held the white piglet in her ...",book4_dorothy_and_the_wizard_in_ozpg22566_images,279.0,19.0,The Wizard Performs Another Trick,8.0,"[279, 280, 281]",dorothy_and_the_wizard_in_oz,as the princess held the white piglet in her a...,2
207,When do we first see a gun in the series?,70,1,"“‘What are you, a man or a pumpkin?’ he asked,...",book2_the_marvelous_land_of_oz_pg54_images,61.0,6.0,Jack Pumpkinhead’s Ride To The Emerald City,8.0,"[61, 62, 63]",the_marvelous_land_of_oz,what are you a man or a pumpkin? he asked poli...,1
208,When do we first see a gun in the series?,70,2,“Followed by her Army the General now rushed t...,book2_the_marvelous_land_of_oz_pg54_images,91.0,8.0,Gen. Jinjur’s Army Of Revolt,9.0,"[91, 92, 93]",the_marvelous_land_of_oz,followed by her army the general now rushed to...,2


In [10]:
# Extract questions that need to be rerun
rerun_questions = get_unique_null_questions(master_df)
rerun_questions

array(['How old is the Scarecrow when Dorothy finds him?',
       'When do Dorothy and Ozma meet?', 'How do you wind Tiktok?',
       'How does Dorothy return to Kansas in book 4, Dorothy and the Wizard in Oz?',
       'In The Emerald City of Oz, what is the first instance of the author speaking in second person? ',
       'How is Mogodore defeated?'], dtype=object)

In [11]:
#Read in continuity checked results for run 4
df5_4 = pd.read_csv('/work/oz_extractor_5_4_1_is_match_results_CONT_CHECKED_2.csv')

#Add column for run number
df5_4['run_number'] = 4

df5_4.head()


,question,question_number,result_rank,excerpt,filename,model_chunk_index,model_chapter_number,model_chapter_title,model_chunk_in_chapter_index,all_model_chunks,model_filename,excerpt_cleaned,run_number
0,How old is the Scarecrow when Dorothy finds him?,1,1,"“There was a great cornfield beyond the fence,...",book1_the_wonderful_wizard_of_oz_pg55_images,28.0,3.0,How Dorothy Saved The Scarecrow,6.0,"[28, 29, 30]",the_wonderful_wizard_of_oz,there was a great cornfield beyond the fence a...,4
1,How old is the Scarecrow when Dorothy finds him?,1,2,"“Originally he was a farmer, for he passed his...",book6_the_emerald_city_of_oz_pg41667_images,NaN,NaN,NaN,NaN,[],NaN,originally he was a farmer for he passed his e...,4
2,How old is the Scarecrow when Dorothy finds him?,1,3,"“And how could it be otherwise, with so many u...",book23_jack_pumpkinhead.,210.0,17.0,The Capture Of The Emerald City,0.0,"[210, 211, 212]",jack_pumpkinhead_of_oz,and how could it be otherwise with so many unu...,4
3,When do Dorothy and Ozma meet?,2,1,"""DOROTHY GALE was sitting in one of her rooms ...",book7_the_patchwork_girl_of_oz_pg32094_images,228.0,16.0,Princess Dorothy,0.0,[228],the_patchwork_girl_of_oz,dorothy gale was sitting in one of her rooms i...,4
4,When do Dorothy and Ozma meet?,2,2,"""That night Dorothy slept in a pleasant little...",book3_ozma_of_oz_pg33361_images,NaN,NaN,NaN,NaN,[],NaN,that night dorothy slept in a pleasant little ...,4


In [12]:
# Replace hallucinated excerpts in master_df with those from df5_4
master_df = replace_hallucinated_excerpts(master_df, df5_4)
master_df

,question,question_number,result_rank,excerpt,filename,model_chunk_index,model_chapter_number,model_chapter_title,model_chunk_in_chapter_index,all_model_chunks,model_filename,excerpt_cleaned,run_number
0,What color are Dorothy's shoes?,1,1,"“Dorothy had only one other dress, but that ha...",book1_the_wonderful_wizard_of_oz_pg55_images,22.0,3.0,How Dorothy Saved The Scarecrow,0.0,"[22, 23]",the_wonderful_wizard_of_oz,dorothy had only one other dress but that happ...,2
1,What color are Dorothy's shoes?,1,2,"“Dorothy looked, and gave a little cry of frig...",book1_the_wonderful_wizard_of_oz_pg55_images,12.0,2.0,The Council With The Munchkins,3.0,"[12, 13]",the_wonderful_wizard_of_oz,dorothy looked and gave a little cry of fright...,2
2,What color are Dorothy's shoes?,1,3,“‘Your Silver Shoes will carry you over the de...,book1_the_wonderful_wizard_of_oz_pg55_images,252.0,23.0,Glinda The Good Witch Grants Dorothy’s Wish,5.0,"[252, 253]",the_wonderful_wizard_of_oz,your silver shoes will carry you over the dese...,2
3,How old is the Scarecrow when Dorothy finds him?,2,1,"“She ate a hearty breakfast, and watched a wee...",book1_the_wonderful_wizard_of_oz_pg55_images,27.0,3.0,How Dorothy Saved The Scarecrow,5.0,"[27, 28, 29, 30]",the_wonderful_wizard_of_oz,she ate a hearty breakfast and watched a wee m...,1
4,How old is the Scarecrow when Dorothy finds him?,2,2,“Dorothy reached up both arms and lifted the f...,book1_the_wonderful_wizard_of_oz_pg55_images,30.0,3.0,How Dorothy Saved The Scarecrow,8.0,"[30, 31, 32, 33]",the_wonderful_wizard_of_oz,dorothy reached up both arms and lifted the fi...,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
205,Why doesn't Eureka immediately tell Ozma where...,69,2,"""The piglet that belonged to the Princess wore...",book4_dorothy_and_the_wizard_in_ozpg22566_images,280.0,19.0,The Wizard Performs Another Trick,9.0,"[280, 281, 282, 283]",dorothy_and_the_wizard_in_oz,the piglet that belonged to the princess wore ...,2
206,Why doesn't Eureka immediately tell Ozma where...,69,3,"""As the Princess held the white piglet in her ...",book4_dorothy_and_the_wizard_in_ozpg22566_images,279.0,19.0,The Wizard Performs Another Trick,8.0,"[279, 280, 281]",dorothy_and_the_wizard_in_oz,as the princess held the white piglet in her a...,2
207,When do we first see a gun in the series?,70,1,"“‘What are you, a man or a pumpkin?’ he asked,...",book2_the_marvelous_land_of_oz_pg54_images,61.0,6.0,Jack Pumpkinhead’s Ride To The Emerald City,8.0,"[61, 62, 63]",the_marvelous_land_of_oz,what are you a man or a pumpkin? he asked poli...,1
208,When do we first see a gun in the series?,70,2,“Followed by her Army the General now rushed t...,book2_the_marvelous_land_of_oz_pg54_images,91.0,8.0,Gen. Jinjur’s Army Of Revolt,9.0,"[91, 92, 93]",the_marvelous_land_of_oz,followed by her army the general now rushed to...,2


In [13]:
# Extract questions that need to be rerun
rerun_questions = get_unique_null_questions(master_df)
rerun_questions

array(['When do Dorothy and Ozma meet?',
       'In The Emerald City of Oz, what is the first instance of the author speaking in second person? '],
      dtype=object)

In [14]:
# Read in  continuity checked results for run 5
df5_5 = pd.read_csv('/work/oz_extractor_5_5_is_match_results_CONT_CHECKED_1.csv')

#Add column for run number
df5_5['run_number'] = 5

df5_5.head()

,question,question_number,result_rank,excerpt,filename,model_chunk_index,model_chapter_number,model_chapter_title,model_chunk_in_chapter_index,all_model_chunks,model_filename,excerpt_cleaned,run_number
0,When do Dorothy and Ozma meet?,1,1,"""Several days of festivity and merry-making fo...",book4_dorothy_and_the_wizard_in_ozpg22566_images,NaN,NaN,NaN,NaN,[],NaN,several days of festivity and merry making fol...,5
1,When do Dorothy and Ozma meet?,1,2,"""That night Dorothy slept in a pleasant little...",book3_ozma_of_oz_pg33361_images,NaN,NaN,NaN,NaN,[],NaN,that night dorothy slept in a pleasant little ...,5
2,When do Dorothy and Ozma meet?,1,3,"""These consisted of a beautiful sitting room, ...",book6_the_emerald_city_of_oz_pg41667_images,55.0,5.0,How Dorothy Became A Princess,7.0,"[55, 56]",the_emerald_city_of_oz,these consisted of a beautiful sitting room a ...,5
3,"In The Emerald City of Oz, what is the first i...",2,1,"""It has nine thousand, six hundred and fifty-f...",book6_the_emerald_city_of_oz_pg41667_images,24.0,3.0,How Ozma Granted Dorothy's Request,0.0,"[24, 25, 26, 27, 28]",the_emerald_city_of_oz,it has nine thousand six hundred and fifty fou...,5
4,"In The Emerald City of Oz, what is the first i...",2,2,"""There were many tailors and dressmakers and s...",book6_the_emerald_city_of_oz_pg41667_images,NaN,NaN,NaN,NaN,[],NaN,there were many tailors and dressmakers and sh...,5


In [15]:
# Replace hallucinated excerpts in master_df with those from df5_5
master_df = replace_hallucinated_excerpts(master_df, df5_5)
master_df.head()

,question,question_number,result_rank,excerpt,filename,model_chunk_index,model_chapter_number,model_chapter_title,model_chunk_in_chapter_index,all_model_chunks,model_filename,excerpt_cleaned,run_number
0,What color are Dorothy's shoes?,1,1,"“Dorothy had only one other dress, but that ha...",book1_the_wonderful_wizard_of_oz_pg55_images,22.0,3.0,How Dorothy Saved The Scarecrow,0.0,"[22, 23]",the_wonderful_wizard_of_oz,dorothy had only one other dress but that happ...,2
1,What color are Dorothy's shoes?,1,2,"“Dorothy looked, and gave a little cry of frig...",book1_the_wonderful_wizard_of_oz_pg55_images,12.0,2.0,The Council With The Munchkins,3.0,"[12, 13]",the_wonderful_wizard_of_oz,dorothy looked and gave a little cry of fright...,2
2,What color are Dorothy's shoes?,1,3,“‘Your Silver Shoes will carry you over the de...,book1_the_wonderful_wizard_of_oz_pg55_images,252.0,23.0,Glinda The Good Witch Grants Dorothy’s Wish,5.0,"[252, 253]",the_wonderful_wizard_of_oz,your silver shoes will carry you over the dese...,2
3,How old is the Scarecrow when Dorothy finds him?,2,1,"“She ate a hearty breakfast, and watched a wee...",book1_the_wonderful_wizard_of_oz_pg55_images,27.0,3.0,How Dorothy Saved The Scarecrow,5.0,"[27, 28, 29, 30]",the_wonderful_wizard_of_oz,she ate a hearty breakfast and watched a wee m...,1
4,How old is the Scarecrow when Dorothy finds him?,2,2,“Dorothy reached up both arms and lifted the f...,book1_the_wonderful_wizard_of_oz_pg55_images,30.0,3.0,How Dorothy Saved The Scarecrow,8.0,"[30, 31, 32, 33]",the_wonderful_wizard_of_oz,dorothy reached up both arms and lifted the fi...,2


In [16]:
# Extract questions that need to be rerun
rerun_questions = get_unique_null_questions(master_df)
rerun_questions

array([], dtype=object)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=f3468430-3cb1-4d6e-8fbc-21329270417f' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>